In [3]:
!git clone https://github.com/Antonomaz/Corpus.git 2> nul || cd Corpus && git pull

Already up to date.


In [4]:
from bs4 import BeautifulSoup
from pprint import pprint
from glob import glob
import json
from numpy import mean, std, amax, amin
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

def is_LGERM(word):
	return word in mots_LGERM

def is_LGERM_phrase(phrase):
	return sum(is_LGERM(word) for word in phrase.split())

def ttr_line(phrase):
	return mean([is_LGERM(word) for word in phrase.split()])

def ttr_page(page):
	return [ttr_line(line) for line in page]





with open("LGERM.json", encoding="utf-8") as f:
    LGERM = json.load(f)
mots_LGERM = set(LGERM)

path = "Corpus/Mazarinades/[1-9]*/*.xml"

files = glob(path)

print(len(files))


lstttrpages = []
for file in tqdm(files):
	with open(file, "r", encoding="utf-8") as f:
		soup = BeautifulSoup(f, features="xml")

	head = soup.find("teiHeader")


	body = soup.find("text").find("body").find("p")

	# Create a list of lines, each line is separed by a <lb/> tag or inside a <l> tag

	lines = []

	for child in body.children:
		if child.name == "l":
			lines.append(child.text)
		elif child.name == "pb":
			lines.append(f"$$$Page_{child['n']}$$$")


	lines = [line.strip() for line in lines if line not in ["\n", ""]]




	pages = []
	for line in lines:

		if line.__contains__("FIN"):
			continue
		if line.startswith("$$$Page_"):
			if len(pages) > 0 and not pages[-1]:
				continue
			pages.append([])
			continue

		pages[-1].append(line)



	#lgrempages = [[(len(line.split()), is_LGERM_phrase(line)) for line in page] for page in pages]


	ttrpages = [ttr_page(page) for page in pages]

	lstttrpages.append(ttrpages)

2839


  0%|          | 0/2839 [00:00<?, ?it/s]

FeatureNotFound: Couldn't find a tree builder with the features you requested: xml. Do you need to install a parser library?

In [ ]:
lstttrpages[2]

In [ ]:
lstttrpages

In [ ]:
# construct some data like what you have:
#x = np.array(lstttrpages)

min = lambda x: np.min(x, axis=0)
max = lambda x: np.max(x, axis=0)
mean = lambda x: np.mean(x, axis=0)
std = lambda x: np.std(x, axis=0)

x = np.array((min(e), max(e), mean(e), std(e)) for e in lstttrpages)

# create stacked errorbars:
plt.errorbar(np.arange(2), means, std, fmt='ok', lw=3)
plt.errorbar(np.arange(2), means, [means - mins, maxes - means],
             fmt='.k', ecolor='gray', lw=1)

plt.show()

In [ ]:
lstttrpages

In [ ]:
# Create a bar plot showing the statistics for each page
n_pages = len(lstttrpages[0])
mean_ttr = np.zeros(n_pages)
std_ttr = np.zeros(n_pages)
min_ttr = np.zeros(n_pages)
max_ttr = np.zeros(n_pages)

for i in range(n_pages):
    page_ttr = [lstttrpages[j][i] for j in range(len(lstttrpages))]
    mean_ttr[i] = np.mean(page_ttr)
    std_ttr[i] = np.std(page_ttr)
    min_ttr[i] = np.min(page_ttr)
    max_ttr[i] = np.max(page_ttr)

page_numbers = np.arange(1, n_pages+1)
width = 0.2
fig, ax = plt.subplots()
ax.bar(page_numbers - width, mean_ttr, width, label='Mean')
ax.bar(page_numbers, std_ttr, width, label='Std Dev')
ax.bar(page_numbers + width, max_ttr, width, label='Max')
ax.bar(page_numbers + 2*width, min_ttr, width, label='Min')
ax.legend()
ax.set_xlabel('Page Number')
ax.set_ylabel('TTR')
plt.show()

In [ ]:
list(x.tolist())

In [ ]:
import matplotlib.pyplot as plt

plt.plot(ttrpages)
plt.show()
